In [2]:

from config.config import get_cfg_defaults
from model import generate_model
import torch
import torch.nn.functional as F
import torch.nn as nn
from einops import rearrange, repeat
from data_loader import iterator_factory_augmentResNet
from loss import create_loss
from loss import metric
from datetime import datetime
from torch.optim import SGD, lr_scheduler
import os
import numpy as np

class Combine(nn.Module):

    def __init__(self, cfg):
        super().__init__()
        self.backbone = generate_model(cfg.BACKBONE)

        # for param in self.backbone.parameters():
        #     param.requires_grad = False

        # self.head = generate_model(cfg.HEAD)

        self.file_name = cfg.TRAIN.MODEL_NAME

    def forward(self, data):

        B, *_ = data.shape

        data = rearrange(data, 'b s c t h w -> (b s) c t h w')

        # feature: ((b s), c, t, h, w)
        # pred: ((b s), n)
        features, pred = self.backbone(data)

        # features = rearrange(features, '(b s) c t h w -> b s c t h w', b = B)

        # features: (b, s, c, t, h, w)
        # outputs = self.head(features)
        return pred

    def save_model(self, file_path, epoch):
        file_name = f"{file_path}-epoch:{epoch}.pth"
        torch.save(self.state_dict(), file_name)
        pass

    def load_model(self, file_path):
        result = torch.load(file_path)
        pass

def load_checkpoint(cfg ,model):

    checkpoint = torch.load(cfg.TRAIN.PRETRAIN_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    epoch = checkpoint['epoch']

    try:
        best_acc = checkpoint['best_acc']
    except:
        best_acc = 0

    return epoch, best_acc


In [3]:
cfg = get_cfg_defaults()
config_file = 'log/23_06_13-even_crop_random:1-resnet3d_50--video_per:0.3-video_len16-optimize:SGD-loss:crossentropylossmean/config.yaml'
pretrain_path = 'log/23_06_13-even_crop_random:1-resnet3d_50--video_per:0.3-video_len16-optimize:SGD-loss:crossentropylossmean/model_best.pth'

cfg.merge_from_file(config_file)

# cho pretrain_path
cfg.TRAIN.PRETRAIN_PATH = pretrain_path

cfg.DATA.EVAL_ONLY = True
# đưa batch size về worker về 1 hết
cfg.DATA.BATCH_SIZE = 1
cfg.DATA.WORKERS = 0
cfg.TRAIN.BATCH_SIZE = 1
cfg.TRAIN.WORKERS = 0
cfg.freeze()


model = Combine(cfg).to(cfg.TRAIN.DEVICE)
a,b = None, None
_, _ = load_checkpoint(cfg, model)

import json

class_dict = open('label/UCF-101/dictionary.json')
class_dict = json.load(class_dict)

class_dict_new = {}

for key, item in class_dict.items():
    class_dict_new[item] = key


In [7]:
val_loader, len_video_val = iterator_factory_augmentResNet.create(cfg.DATA)
if cfg.TRAIN.LOSS == 'crossentropylossmean':
    criterion = create_loss.CrossEntropyLossMean
elif cfg.TRAIN.LOSS == 'crossentropyloss':
    criterion = create_loss.CrossEntropyLoss

num_test = 2
output_test_list = []
video_path_test_list = []
sampled_indices_list_list = []

model.eval()
# evaluation
with torch.no_grad():

    accuracy_val = 0
    loss_val = 0

    for i, (data, targets, video_path) in enumerate(val_loader):
        print(data.shape)
        if num_test < 0:
            break
        num_test -= 1

        video_path_test_list.append(video_path)

        if i == len(val_loader) - 1:
            hihi = 0
        data = data.cuda()
        targets = targets.cuda()

        video = (data[0, 0, ...]*255).detach().cpu().numpy().transpose(1,2,3,0).astype(np.uint8)
        xxx = video[0]
        # zero the parameter gradients
        outputs = model(data)
        output_test_list.append(outputs)
        outputs = rearrange(outputs, '(b s) c -> b s c', s = cfg.DATA.NUM_SAMPLERS)
        loss = criterion(outputs, targets)

        # calculate metric here
        # val_metric.update(outputs, targets, loss)


    # write in file
    # val_metric.write_file(0)


2023-06-20 23:29:03 it-System-Product-Name root[546769] INFO VideoIter:: Found dict at: label/UCF-101/dictionary.json 

2023-06-20 23:29:03 it-System-Product-Name root[546769] INFO VideoIter:: iterator initialized (phase: 'label/UCF-101/val.csv', num: 3783)


torch.Size([1, 1, 3, 16, 112, 112])
torch.Size([1, 1, 3, 16, 112, 112])
torch.Size([1, 1, 3, 16, 112, 112])
torch.Size([1, 1, 3, 16, 112, 112])


In [8]:
predict_list = []

for i in range(len(output_test_list)):
    _, indices = output_test_list[i].topk(1)

    top_k = int(indices)

    predict_list.append(class_dict_new[top_k])

In [9]:
import os 
import shutil
def delete_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for filename in os.listdir(folder_name):
        file_path = os.path.join(folder_name, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

delete_folder('video_test')

import glob   
import cv2
# đọc video   
for i in range(len(video_path_test_list)):
    video_path = video_path_test_list[i]
    img_list = glob.glob(f'{video_path[0]}/*.jpg')
    img_list.sort()

    hf, wf, _ = cv2.imread(img_list[0]).shape

    result = cv2.VideoWriter(f"video_test/{video_path[0].split('/')[-1]}.mp4",
                cv2.VideoWriter_fourcc(*'MP4V'),
                10, (wf, hf))
    
    label = video_path[0].split('/')[-2] 
    predict = predict_list[i]

    for img in img_list:
        frame = cv2.imread(img)

        frame = cv2.putText(frame, f'Label: {label}', (0, 20), cv2.FONT_ITALIC, 0.5, 
                    (0, 255, 0), 1, cv2.LINE_AA, False)

        frame = cv2.putText(frame, f'Predict: {predict}', (0, 60), cv2.FONT_ITALIC, 0.5, 
                    (0, 0, 255), 1, cv2.LINE_AA, False)

        result.write(frame)
    print(frame.shape)
    
    result.release()
    


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


(240, 320, 3)
(240, 320, 3)
(240, 320, 3)


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
